# Snowflake Cortex Search BYOV Demo: Image Similarity Search

This notebook demonstrates how to build an image similarity search system using Snowflake's Cortex Search with "Bring Your Own Vector" (BYOV) capabilities. We'll create a pipeline that:

1. Generates demo product images from Unsplash
2. Stores them in Snowflake tables and stages  
3. Creates vector embeddings using AI_EMBED
4. Sets up a Cortex Search service for similarity queries

## 1. Setup and Dependencies

Import required libraries and set up table configuration.


### Environment Detection and Session Creation

Detect whether we're running in Snowflake native environment or locally, then create the appropriate Snowpark session.


In [1]:
import snowflake.connector
import pandas as pd
import os
import getpass

TABLE_NAME = "IMAGES_TABLE"



### Database and Warehouse Configuration

Configure the Snowflake session to use the correct database, schema, and warehouse for our demo.


In [2]:
# Detect environment first - check if we're running in Snowflake
def detect_snowflake_environment():
    """Detect if we're running inside Snowflake or locally"""
    try:
        # Check if snowflake.snowpark.context is available (Snowflake native)
        import snowflake.snowpark.context
        # Also check if we can actually get an active session
        from snowflake.snowpark.context import get_active_session
        test_session = get_active_session()
        return True
    except:
        return False

# Determine environment and create session
IS_SNOWFLAKE_NATIVE = detect_snowflake_environment()

if IS_SNOWFLAKE_NATIVE:
    # Running in Snowflake - use native session
    print("🚀 Detected Snowflake native environment")
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    ENVIRONMENT = "SNOWFLAKE_NATIVE"
    print("✅ Using Snowflake native session (Container Runtime)")
else:
    # Running locally - need to create session with connection parameters
    print("💻 Detected local development environment")
    from snowflake.snowpark import Session
    
    # Try to get existing session first
    try:
        session = Session.builder.getOrCreate()
        print("✅ Using existing Snowflake session")
        ENVIRONMENT = "LOCAL_DEV"
    except:
        # No existing session - need to create one with connection config
        print("⚠️ No existing session found. Creating new session...")
        print("📋 Note: For local development, you need to configure Snowflake connection.")
        print("   You can do this by setting up ~/.snowflake/connections.toml")
        print("   or by providing connection parameters programmatically.")
        
        # Create session with connection parameters (will use default connection if available)
        session = Session.builder.create()
        ENVIRONMENT = "LOCAL_DEV"
        print("✅ Created new local development session")

# Display connection info
print(f"\n🎯 Environment: {ENVIRONMENT}")
print(f"🏠 Current database: {session.get_current_database()}")
print(f"📊 Current schema: {session.get_current_schema()}")
print(f"🏗️ Current warehouse: {session.get_current_warehouse()}")
print(f"👤 Current role: {session.get_current_role()}")


💻 Detected local development environment
✅ Using existing Snowflake session

🎯 Environment: LOCAL_DEV
🏠 Current database: "CC_ML_JOBS"
📊 Current schema: "PUBLIC"
🏗️ Current warehouse: "COMPUTE_WH"
👤 Current role: "SPCS_PSE_ROLE"


In [3]:
# Database and warehouse setup using Snowpark session
setup_commands = [
    "CREATE OR REPLACE DATABASE CC_IMAGES_POC",
    "USE SCHEMA PUBLIC",
    "USE WAREHOUSE COMPUTE_WH"
]

for cmd in setup_commands:
    print(f"✅ Executing: {cmd}")
    session.sql(cmd).collect()
    print(f"✅ Executed: {cmd}")

print("\n🎯 Database and warehouse setup completed")

✅ Executing: CREATE OR REPLACE DATABASE CC_IMAGES_POC
✅ Executed: CREATE OR REPLACE DATABASE CC_IMAGES_POC
✅ Executing: USE SCHEMA PUBLIC
✅ Executed: USE SCHEMA PUBLIC
✅ Executing: USE WAREHOUSE COMPUTE_WH
✅ Executed: USE WAREHOUSE COMPUTE_WH

🎯 Database and warehouse setup completed


## 2. Generate Demo Product Images

Function to generate a curated collection of 100+ product images across multiple categories (Electronics, Fashion, Home, Beauty, Sports) from Unsplash for reliable testing.



In [4]:
# Install additional packages for web scraping

import requests
from bs4 import BeautifulSoup
import time
import random
from urllib.parse import urljoin, urlparse
import pandas as pd


In [5]:
def generate_product_images(num_images=100):
    """
    Generate a comprehensive list of demo product image URLs for testing
    Uses high-quality product images from Unsplash and other reliable sources
    
    Args:
        num_images: Number of image URLs to generate (default: 100)
    
    Returns:
        DataFrame with IMAGE_URL column containing product images
    """
    
    # Expanded collection of product image URLs from various categories
    base_demo_urls = [
        # Electronics & Technology (20 images)
        "https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=600",  # Headphones
        "https://images.unsplash.com/photo-1511707171634-5f897ff02aa9?w=600",  # Phone
        "https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=600",  # Laptop
        "https://images.unsplash.com/photo-1567070734993-6b6d2b7bdac6?w=600",  # Smartphone
        "https://images.unsplash.com/photo-1550009158-9ebf69173e03?w=600",  # Camera
        "https://images.unsplash.com/photo-1593642702821-c8da6771f0c6?w=600",  # Gaming
        "https://images.unsplash.com/photo-1484704849700-f032a568e944?w=600",  # Tablet
        "https://images.unsplash.com/photo-1498049794561-7780e7231661?w=600",  # Desktop
        "https://images.unsplash.com/photo-1583394838336-acd977736f90?w=600",  # Earbuds
        "https://images.unsplash.com/photo-1572569511254-d8f925fe2cbb?w=600",  # Smart watch
        "https://images.unsplash.com/photo-1518444065439-e933c06ce9cd?w=600",  # VR headset
        "https://images.unsplash.com/photo-1550745165-9bc0b252726f?w=600",  # Gaming console
        "https://images.unsplash.com/photo-1586953208448-b95a79798f07?w=600",  # Drone
        "https://images.unsplash.com/photo-1609205264155-797fb74e6b3d?w=600",  # Keyboard
        "https://images.unsplash.com/photo-1527864550417-7fd91fc51a46?w=600",  # Mouse
        "https://images.unsplash.com/photo-1558618047-3c8c76ca7d13?w=600",  # Speakers
        "https://images.unsplash.com/photo-1507003211169-0a1dd7228f2d?w=600",  # Smart home
        "https://images.unsplash.com/photo-1571902943202-507ec2618e8f?w=600",  # Hard drive
        "https://images.unsplash.com/photo-1593320215240-3c81b82b7551?w=600",  # Router
        "https://images.unsplash.com/photo-1517336714731-489689fd1ca8?w=600",  # MacBook
        
        # Fashion & Clothing (25 images)
        "https://images.unsplash.com/photo-1523381210434-271e8be1f52b?w=600",  # Clothing
        "https://images.unsplash.com/photo-1515886657613-9f3515b0c78f?w=600",  # Fashion
        "https://images.unsplash.com/photo-1542291026-7eec264c27ff?w=600",  # Sneakers
        "https://images.unsplash.com/photo-1549298916-b41d501d3772?w=600",  # Dress shoes
        "https://images.unsplash.com/photo-1556905055-8f358a7a47b2?w=600",  # Sunglasses
        "https://images.unsplash.com/photo-1434389677669-e08b4cac3105?w=600",  # Dress
        "https://images.unsplash.com/photo-1521572163474-6864f9cf17ab?w=600",  # T-shirt
        "https://images.unsplash.com/photo-1603252109303-2751441b4e54?w=600",  # Jacket
        "https://images.unsplash.com/photo-1506629905607-c65037d39b47?w=600",  # Jeans
        "https://images.unsplash.com/photo-1544966503-7cc5ac882d5f?w=600",  # Handbag
        "https://images.unsplash.com/photo-1553062407-98eeb64c6a62?w=600",  # Hat
        "https://images.unsplash.com/photo-1571945153237-4929e783af4a?w=600",  # Boots
        "https://images.unsplash.com/photo-1582142306909-195724d33cb0?w=600",  # Sweater
        "https://images.unsplash.com/photo-1594633312681-425c7b97ccd1?w=600",  # Scarf
        "https://images.unsplash.com/photo-1572804013309-59a88b7e92f1?w=600",  # Belt
        "https://images.unsplash.com/photo-1618354691373-d851c5c3a990?w=600",  # Backpack
        "https://images.unsplash.com/photo-1571181520189-2ca222fddcbe?w=600",  # Suit
        "https://images.unsplash.com/photo-1605518216938-7c31b7b14ad0?w=600",  # Shirt
        "https://images.unsplash.com/photo-1607345366928-199ea26cfe3e?w=600",  # Shoes
        "https://images.unsplash.com/photo-1594223274512-ad4803739b7c?w=600",  # Polo
        "https://images.unsplash.com/photo-1581044777550-4cfa60707c03?w=600",  # Hoodie
        "https://images.unsplash.com/photo-1582004341910-a1a1ec8a7501?w=600",  # Skirt
        "https://images.unsplash.com/photo-1584380931214-dbb5b72e7fd0?w=600",  # Blouse
        "https://images.unsplash.com/photo-1588117472013-59bb13edafec?w=600",  # Pants
        "https://images.unsplash.com/photo-1620799140408-edc6dcb6d633?w=600",  # Formal wear
        
        # Home & Furniture (20 images)
        "https://images.unsplash.com/photo-1586023492125-27b2c045efd7?w=600",  # Furniture
        "https://images.unsplash.com/photo-1555041469-a586c61ea9bc?w=600",  # Home decor
        "https://images.unsplash.com/photo-1507003211169-0a1dd7228f2d?w=600",  # Kitchen
        "https://images.unsplash.com/photo-1555446226-75e63ad2b842?w=600",  # Dining table
        "https://images.unsplash.com/photo-1567538096630-e0c55bd6374c?w=600",  # Bedroom
        "https://images.unsplash.com/photo-1554995207-c18c203602cb?w=600",  # Living room
        "https://images.unsplash.com/photo-1616486338812-3dadae4b4ace?w=600",  # Sofa
        "https://images.unsplash.com/photo-1571898829755-fcf5b8f6bb13?w=600",  # Chair
        "https://images.unsplash.com/photo-1506439773649-6e0eb8cfb237?w=600",  # Lamp
        "https://images.unsplash.com/photo-1501594907352-04cda38ebc29?w=600",  # Vase
        "https://images.unsplash.com/photo-1519947486511-46149fa0a254?w=600",  # Cushions
        "https://images.unsplash.com/photo-1553688738-a278b9f063c8?w=600",  # Rug
        "https://images.unsplash.com/photo-1519947486511-46149fa0a254?w=600",  # Curtains
        "https://images.unsplash.com/photo-1582582621959-48d27397dc69?w=600",  # Mirrors
        "https://images.unsplash.com/photo-1586444248902-2f64eddc13df?w=600",  # Bookshelf
        "https://images.unsplash.com/photo-1524758631624-e2822e304c36?w=600",  # Desk
        "https://images.unsplash.com/photo-1549497538-303791108f95?w=600",  # Bed
        "https://images.unsplash.com/photo-1515444744401-0cfd5c63edc2?w=600",  # Wardrobe
        "https://images.unsplash.com/photo-1571508601891-ca5e7a713859?w=600",  # Coffee table
        "https://images.unsplash.com/photo-1555426047-53f0e909ff96?w=600",  # Kitchen appliances
        
        # Beauty & Health (15 images)
        "https://images.unsplash.com/photo-1596462502278-27bfdc403348?w=600",  # Cosmetics
        "https://images.unsplash.com/photo-1512496015851-a90fb38ba796?w=600",  # Skincare
        "https://images.unsplash.com/photo-1559056199-641a0ac8b55e?w=600",  # Perfume
        "https://images.unsplash.com/photo-1570172619644-dfd03ed5d881?w=600",  # Makeup
        "https://images.unsplash.com/photo-1585652757141-4d90e0b21a0e?w=600",  # Lipstick
        "https://images.unsplash.com/photo-1631729371254-42c2892e6e9b?w=600",  # Hair care
        "https://images.unsplash.com/photo-1608248543803-ba4f8c70ae0b?w=600",  # Face cream
        "https://images.unsplash.com/photo-1571513722275-4b41940f54b8?w=600",  # Nail polish
        "https://images.unsplash.com/photo-1575877281692-5a51528abcb1?w=600",  # Soap
        "https://images.unsplash.com/photo-1612817288484-6f916006741a?w=600",  # Shampoo
        "https://images.unsplash.com/photo-1602861440244-b5d85398a1b8?w=600",  # Sunscreen
        "https://images.unsplash.com/photo-1621607295805-1699fa6ff18c?w=600",  # Wellness
        "https://images.unsplash.com/photo-1556228852-80de77fde541?w=600",  # Essential oils
        "https://images.unsplash.com/photo-1623919734507-91ad8ee440c7?w=600",  # Face mask
        "https://images.unsplash.com/photo-1594736797933-d0d4769bab46?w=600",  # Beauty tools
        
        # Sports & Fitness (20 images)
        "https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=600",  # Sports equipment
        "https://images.unsplash.com/photo-1558618047-3c8c76ca7d13?w=600",  # Fitness
        "https://images.unsplash.com/photo-1544966503-7cc5ac882d5f?w=600",  # Yoga mat
        "https://images.unsplash.com/photo-1517838277536-f5f99be501cd?w=600",  # Dumbbells
        "https://images.unsplash.com/photo-1599058918133-58ba3a9dd8eb?w=600",  # Running shoes
        "https://images.unsplash.com/photo-1551698618-1dfe5d97d256?w=600",  # Basketball
        "https://images.unsplash.com/photo-1578662996442-48f60103fc96?w=600",  # Soccer ball
        "https://images.unsplash.com/photo-1628260412297-a3377e45006f?w=600",  # Tennis racket
        "https://images.unsplash.com/photo-1578328819058-b69f3a3b0f6b?w=600",  # Bicycle
        "https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=600",  # Helmet
        "https://images.unsplash.com/photo-1593080358201-edf2897e6a38?w=600",  # Water bottle
        "https://images.unsplash.com/photo-1544966503-7cc5ac882d5f?w=600",  # Resistance bands
        "https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=600",  # Golf clubs
        "https://images.unsplash.com/photo-1578662996442-48f60103fc96?w=600",  # Skateboard
        "https://images.unsplash.com/photo-1599058918133-58ba3a9dd8eb?w=600",  # Gym bag
        "https://images.unsplash.com/photo-1628260412297-a3377e45006f?w=600",  # Protein shaker
        "https://images.unsplash.com/photo-1578328819058-b69f3a3b0f6b?w=600",  # Workout gear
        "https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=600",  # Boxing gloves
        "https://images.unsplash.com/photo-1593080358201-edf2897e6a38?w=600",  # Fitness tracker
        "https://images.unsplash.com/photo-1544966503-7cc5ac882d5f?w=600",   # Swimming goggles
        
        # Additional Electronics & Technology (20 more images)
        "https://images.unsplash.com/photo-1593305841991-05c297ba4575?w=600",  # Microphone
        "https://images.unsplash.com/photo-1573511860302-28c524319d2a?w=600",  # Monitor
        "https://images.unsplash.com/photo-1555617981-dac3880eac6e?w=600",  # Smart TV
        "https://images.unsplash.com/photo-1587829741301-dc798b83add3?w=600",  # Webcam
        "https://images.unsplash.com/photo-1579952363873-27d3bfad9c0d?w=600",  # Laptop stand
        "https://images.unsplash.com/photo-1576741787058-c2d93b61de0a?w=600",  # Power bank
        "https://images.unsplash.com/photo-1569591159212-b02ea8a9f239?w=600",  # USB cables
        "https://images.unsplash.com/photo-1517077304055-6e89abbf09b0?w=600",  # External SSD
        "https://images.unsplash.com/photo-1544966503-7cc5ac882d5f?w=600",  # Bluetooth speaker
        "https://images.unsplash.com/photo-1593642634443-44adaa06623a?w=600",  # Graphics card
        "https://images.unsplash.com/photo-1622979135225-d2ba269cf1ac?w=600",  # RAM memory
        "https://images.unsplash.com/photo-1585509345725-95e72df80c68?w=600",  # Motherboard
        "https://images.unsplash.com/photo-1589739900243-493d3a73b86c?w=600",  # CPU processor
        "https://images.unsplash.com/photo-1605236453806-b0cb981c6e26?w=600",  # Gaming chair
        "https://images.unsplash.com/photo-1502920917128-1aa500764cbd?w=600",  # LED lights
        "https://images.unsplash.com/photo-1518709414026-ed4c7efc866b?w=600",  # Server rack
        "https://images.unsplash.com/photo-1582140381751-2f9263abb717?w=600",  # Network switch
        "https://images.unsplash.com/photo-1588414734732-660b07bf4adf?w=600",  # Car charger
        "https://images.unsplash.com/photo-1606107557195-0e29a4b5b4aa?w=600",  # Wireless charger
        "https://images.unsplash.com/photo-1571510046036-f2e22d1c02db?w=600",  # Security camera
        
        # Additional Fashion & Clothing (25 more images)
        "https://images.unsplash.com/photo-1571945153237-4929e783af4a?w=600",  # High heels
        "https://images.unsplash.com/photo-1590649880765-91b1956b8276?w=600",  # Dress collection
        "https://images.unsplash.com/photo-1594633312681-425c7b97ccd1?w=600",  # Winter coat
        "https://images.unsplash.com/photo-1571508601891-ca5e7a713859?w=600",  # Business suit
        "https://images.unsplash.com/photo-1506629905607-c65037d39b47?w=600",  # Casual wear
        "https://images.unsplash.com/photo-1598300042247-d088f8ab3a91?w=600",  # Watch collection
        "https://images.unsplash.com/photo-1616353071891-c75a0b52e9a1?w=600",  # Designer bag
        "https://images.unsplash.com/photo-1592853625511-ad0edcc69c07?w=600",  # Jewelry
        "https://images.unsplash.com/photo-1573408301185-9146fe634ad0?w=600",  # Earrings
        "https://images.unsplash.com/photo-1583743089696-fa5da2bb4e83?w=600",  # Necklace
        "https://images.unsplash.com/photo-1611652022419-a9419f74343d?w=600",  # Bracelet
        "https://images.unsplash.com/photo-1590649880765-91b1956b8276?w=600",  # Ring set
        "https://images.unsplash.com/photo-1578632292335-df3abbb0d586?w=600",  # Wallet
        "https://images.unsplash.com/photo-1564584217132-2271feaeb3c5?w=600",  # Tie collection
        "https://images.unsplash.com/photo-1601924994987-69e26d50dc26?w=600",  # Cufflinks
        "https://images.unsplash.com/photo-1581338834647-b0fb40704e21?w=600",  # Sports bra
        "https://images.unsplash.com/photo-1618932260643-eee4a2f652a6?w=600",  # Yoga pants
        "https://images.unsplash.com/photo-1590736969955-71cc94901144?w=600",  # Running shorts
        "https://images.unsplash.com/photo-1609139003234-2dbf72d29e5a?w=600",  # Tank top
        "https://images.unsplash.com/photo-1617137984095-74e4e5e3613f?w=600",  # Athletic wear
        "https://images.unsplash.com/photo-1594633312681-425c7b97ccd1?w=600",  # Windbreaker
        "https://images.unsplash.com/photo-1559056199-641a0ac8b55e?w=600",  # Formal shoes
        "https://images.unsplash.com/photo-1571945153237-4929e783af4a?w=600",  # Sandals
        "https://images.unsplash.com/photo-1606107557195-0e29a4b5b4aa?w=600",  # Flip flops
        "https://images.unsplash.com/photo-1571902943202-507ec2618e8f?w=600",  # Socks collection
        
        # Additional Home & Furniture (20 more images)
        "https://images.unsplash.com/photo-1586444248902-2f64eddc13df?w=600",  # Office chair
        "https://images.unsplash.com/photo-1540932239986-30128078f3c5?w=600",  # Kitchen island
        "https://images.unsplash.com/photo-1571508601891-ca5e7a713859?w=600",  # Dining chairs
        "https://images.unsplash.com/photo-1616486338812-3dadae4b4ace?w=600",  # Sectional sofa
        "https://images.unsplash.com/photo-1583847268964-b28dc8f51f92?w=600",  # Ottoman
        "https://images.unsplash.com/photo-1616137466211-f939fc48ddb0?w=600",  # Storage cabinet
        "https://images.unsplash.com/photo-1555426047-53f0e909ff96?w=600",  # Kitchen utensils
        "https://images.unsplash.com/photo-1571898829755-fcf5b8f6bb13?w=600",  # Bar stool
        "https://images.unsplash.com/photo-1506439773649-6e0eb8cfb237?w=600",  # Floor lamp
        "https://images.unsplash.com/photo-1582582621959-48d27397dc69?w=600",  # Wall art
        "https://images.unsplash.com/photo-1571508601891-ca5e7a713859?w=600",  # Side table
        "https://images.unsplash.com/photo-1549497538-303791108f95?w=600",  # King bed
        "https://images.unsplash.com/photo-1615529328331-f8917597711f?w=600",  # Mattress
        "https://images.unsplash.com/photo-1564078516393-cf04bd966897?w=600",  # Pillows
        "https://images.unsplash.com/photo-1586023492125-27b2c045efd7?w=600",  # Throw blanket
        "https://images.unsplash.com/photo-1506439773649-6e0eb8cfb237?w=600",  # Ceiling fan
        "https://images.unsplash.com/photo-1571508601891-ca5e7a713859?w=600",  # Nightstand
        "https://images.unsplash.com/photo-1505693416388-ac5ce068fe85?w=600",  # Kitchen cabinets
        "https://images.unsplash.com/photo-1540932239986-30128078f3c5?w=600",  # Refrigerator
        "https://images.unsplash.com/photo-1571508601891-ca5e7a713859?w=600",  # Dining room set
        
        # Additional Beauty & Health (15 more images)
        "https://images.unsplash.com/photo-1571513722275-4b41940f54b8?w=600",  # Foundation
        "https://images.unsplash.com/photo-1570172619644-dfd03ed5d881?w=600",  # Concealer
        "https://images.unsplash.com/photo-1596462502278-27bfdc403348?w=600",  # Mascara
        "https://images.unsplash.com/photo-1631729371254-42c2892e6e9b?w=600",  # Hair serum
        "https://images.unsplash.com/photo-1608248543803-ba4f8c70ae0b?w=600",  # Moisturizer
        "https://images.unsplash.com/photo-1575877281692-5a51528abcb1?w=600",  # Body wash
        "https://images.unsplash.com/photo-1556228852-80de77fde541?w=600",  # Aromatherapy
        "https://images.unsplash.com/photo-1571513722275-4b41940f54b8?w=600",  # Eye cream
        "https://images.unsplash.com/photo-1602861440244-b5d85398a1b8?w=600",  # Anti-aging serum
        "https://images.unsplash.com/photo-1594736797933-d0d4769bab46?w=600",  # Makeup brushes
        "https://images.unsplash.com/photo-1623919734507-91ad8ee440c7?w=600",  # Clay mask
        "https://images.unsplash.com/photo-1621607295805-1699fa6ff18c?w=600",  # Vitamins
        "https://images.unsplash.com/photo-1596462502278-27bfdc403348?w=600",  # Lip balm
        "https://images.unsplash.com/photo-1570172619644-dfd03ed5d881?w=600",  # Blush
        "https://images.unsplash.com/photo-1575877281692-5a51528abcb1?w=600",  # Hand cream
        
        # Additional Sports & Fitness (20 more images)
        "https://images.unsplash.com/photo-1544966503-7cc5ac882d5f?w=600",  # Treadmill
        "https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=600",  # Exercise bike
        "https://images.unsplash.com/photo-1517838277536-f5f99be501cd?w=600",  # Kettlebells
        "https://images.unsplash.com/photo-1544966503-7cc5ac882d5f?w=600",  # Barbell
        "https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=600",  # Weight plates
        "https://images.unsplash.com/photo-1599058918133-58ba3a9dd8eb?w=600",  # Cross trainers
        "https://images.unsplash.com/photo-1551698618-1dfe5d97d256?w=600",  # Baseball
        "https://images.unsplash.com/photo-1578662996442-48f60103fc96?w=600",  # Football
        "https://images.unsplash.com/photo-1628260412297-a3377e45006f?w=600",  # Badminton racket
        "https://images.unsplash.com/photo-1578328819058-b69f3a3b0f6b?w=600",  # Mountain bike
        "https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=600",  # Bike helmet
        "https://images.unsplash.com/photo-1593080358201-edf2897e6a38?w=600",  # Sports bottle
        "https://images.unsplash.com/photo-1544966503-7cc5ac882d5f?w=600",  # Pull-up bar
        "https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=600",  # Ping pong paddle
        "https://images.unsplash.com/photo-1599058918133-58ba3a9dd8eb?w=600",  # Running belt
        "https://images.unsplash.com/photo-1628260412297-a3377e45006f?w=600",  # Volleyball
        "https://images.unsplash.com/photo-1578328819058-b69f3a3b0f6b?w=600",  # Ice skates
        "https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=600",  # Punching bag
        "https://images.unsplash.com/photo-1593080358201-edf2897e6a38?w=600",  # Jump rope
        "https://images.unsplash.com/photo-1544966503-7cc5ac882d5f?w=600"   # Ab roller
    ]
    
    # If we need more images than base collection, cycle through with variations
    demo_urls = []
    variations = ['w=600', 'w=650', 'w=700', 'w=750', 'w=800']  # Size variations
    
    for i in range(num_images):
        base_index = i % len(base_demo_urls)
        variation_index = i // len(base_demo_urls)
        
        base_url = base_demo_urls[base_index]
        if variation_index < len(variations):
            # Replace size parameter with variation
            modified_url = base_url.replace('w=600', variations[variation_index])
        else:
            # Add additional parameter for more variations
            modified_url = base_url + f"&v={variation_index}"
        
        demo_urls.append(modified_url)
    
    print(f"🎭 Generated {len(demo_urls)} demo product image URLs")
    print(f"📊 Categories: Electronics, Fashion, Home, Beauty, Sports")
    
    return pd.DataFrame({'IMAGE_URL': demo_urls})



In [7]:
# Helper function to validate if URLs are accessible (keep this utility function)

# Helper function to validate if URLs are accessible
def validate_image_urls(df, max_check=5):
    """Quick validation to check if URLs are accessible"""
    if len(df) == 0:
        return df
        
    print(f"\n🔍 Validating image URLs (checking first {max_check})...")
    valid_urls = []
    
    for i, url in enumerate(df['IMAGE_URL'].head(max_check)):
        try:
            response = requests.head(url, timeout=5)
            if response.status_code == 200:
                valid_urls.append(url)
                print(f"   ✅ {url} - OK")
            else:
                print(f"   ⚠️ {url} - Status: {response.status_code}")
        except Exception as e:
            print(f"   ❌ {url} - Error: {e}")
    
    # If we validated some URLs, add the rest without checking
    if valid_urls:
        remaining_urls = df['IMAGE_URL'].iloc[max_check:].tolist()
        all_urls = valid_urls + remaining_urls
        return pd.DataFrame({'IMAGE_URL': all_urls})
    else:
        return df


In [8]:
# 🚀 GENERATE 100 DEMO PRODUCT IMAGES
print("🚀 Generating demo product images for testing...")

# Use the expanded demo images function to get 100 images
try:
    print("\n🎭 Creating comprehensive demo product image dataset...")
    df = generate_product_images(num_images=100)
    
    if len(df) > 0:
        print(f"\n✅ Successfully generated {len(df)} demo product image URLs")
        
        # Validate a few sample URLs to ensure they're accessible
        df = validate_image_urls(df, max_check=5)
        
        print(f"\n🎉 Final result: {len(df)} image URLs ready for processing")
        print(f"📍 Source: High-quality demo images from Unsplash")
        print(f"🏷️ Categories: Electronics, Fashion, Home & Furniture, Beauty & Health, Sports & Fitness")
        
        print(f"\n🔗 Sample URLs:")
        for i, url in enumerate(df['IMAGE_URL'].head(5), 1):
            print(f"  {i}. {url}")
    else:
        print("\n❌ Failed to generate demo images")
        df = pd.DataFrame({'IMAGE_URL': []})
        
except Exception as e:
    print(f"\n❌ Error generating demo images: {e}")
    df = pd.DataFrame({'IMAGE_URL': []})


🚀 Generating demo product images for testing...

🎭 Creating comprehensive demo product image dataset...
🎭 Generated 100 demo product image URLs
📊 Categories: Electronics, Fashion, Home, Beauty, Sports

✅ Successfully generated 100 demo product image URLs

🔍 Validating image URLs (checking first 5)...
   ✅ https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=600 - OK
   ✅ https://images.unsplash.com/photo-1511707171634-5f897ff02aa9?w=600 - OK
   ✅ https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=600 - OK
   ⚠️ https://images.unsplash.com/photo-1567070734993-6b6d2b7bdac6?w=600 - Status: 404
   ✅ https://images.unsplash.com/photo-1550009158-9ebf69173e03?w=600 - OK

🎉 Final result: 99 image URLs ready for processing
📍 Source: High-quality demo images from Unsplash
🏷️ Categories: Electronics, Fashion, Home & Furniture, Beauty & Health, Sports & Fitness

🔗 Sample URLs:
  1. https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=600
  2. https://images.unsplash.co

## 3. Create Snowflake Table Schema

Create a clean table structure to store product image URLs and metadata.


In [9]:
# Define table name for product images

# Create simplified table DDL for product image URLs
create_table_sql = f"""
CREATE OR REPLACE TABLE {TABLE_NAME} (
    ID NUMBER AUTOINCREMENT PRIMARY KEY,
    IMAGE_URL VARCHAR(2000) NOT NULL,
    SOURCE_SITE VARCHAR(100) DEFAULT 'demo_images',
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
    PROCESSED_AT TIMESTAMP NULL,
    STATUS VARCHAR(50) DEFAULT 'PENDING'
)
"""

print("📋 Table creation SQL for product images:")
print(create_table_sql)


📋 Table creation SQL for product images:

CREATE OR REPLACE TABLE IMAGES_TABLE (
    ID NUMBER AUTOINCREMENT PRIMARY KEY,
    IMAGE_URL VARCHAR(2000) NOT NULL,
    SOURCE_SITE VARCHAR(100) DEFAULT 'demo_images',
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
    PROCESSED_AT TIMESTAMP NULL,
    STATUS VARCHAR(50) DEFAULT 'PENDING'
)



In [10]:
# Execute table creation
try:
    session.sql(create_table_sql).collect()
    print(f"Table '{TABLE_NAME}' created successfully!")
except Exception as e:
    print(f"Error creating table: {e}")


Table 'IMAGES_TABLE' created successfully!


## 4. Data Preprocessing and Cleaning

Prepare the demo image URLs for upload to Snowflake.


In [11]:
# Clean and prepare demo image URLs for upload
if len(df) > 0:
    df_clean = df.copy()
    
    # Clean and validate image URLs
    df_clean['IMAGE_URL'] = df_clean['IMAGE_URL'].astype(str).str[:2000]  # Limit URL length
    
    # Remove any invalid URLs
    df_clean = df_clean[df_clean['IMAGE_URL'].str.contains('http', na=False)]
    df_clean = df_clean[~df_clean['IMAGE_URL'].str.contains('logo|icon', case=False, na=False)]
    
    # Remove duplicates
    df_clean = df_clean.drop_duplicates(subset=['IMAGE_URL'])
    
    print(f"✅ Data cleaned and validated")
    print(f"📊 Original count: {len(df)}")
    print(f"📊 After cleaning: {len(df_clean)}")
    print(f"📊 Shape: {df_clean.shape}")
    print(f"\n🔗 Sample cleaned URLs:")
    for i, url in enumerate(df_clean['IMAGE_URL'].head(3), 1):
        print(f"  {i}. {url}")
        
else:
    print("⚠️ No data to clean - scraping may have failed")
    df_clean = pd.DataFrame({'IMAGE_URL': []})


✅ Data cleaned and validated
📊 Original count: 99
📊 After cleaning: 85
📊 Shape: (85, 1)

🔗 Sample cleaned URLs:
  1. https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=600
  2. https://images.unsplash.com/photo-1511707171634-5f897ff02aa9?w=600
  3. https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=600


## 5. Upload Data to Snowflake


### Execute Image Download Workflow

Run the complete image download workflow to process all URLs from the table and store the actual image files in the Snowflake stage.


In [12]:
# Upload DataFrame to Snowflake using Snowpark session
try:
    # Use the session that was already established
    # session.write_pandas() is the Snowpark equivalent of write_pandas()
    
    # First truncate the table if it exists
    session.sql(f"TRUNCATE TABLE IF EXISTS {TABLE_NAME}").collect()
    
    # Write the DataFrame using Snowpark session
    df_snowpark = session.write_pandas(
        df_clean,
        table_name=TABLE_NAME,
        auto_create_table=False,  # We already created the table
        overwrite=False,  # We truncated manually above
        chunk_size=1000  # Process in chunks of 1000 rows
    )
    
    # Get the number of rows uploaded
    row_count = session.sql(f"SELECT COUNT(*) FROM {TABLE_NAME}").collect()[0][0]
    
    print(f"Successfully uploaded {row_count} rows to Snowflake table '{TABLE_NAME}'")
    print(f"✅ Data upload completed using Snowpark session")
    
    # Optional: Show the Snowpark DataFrame info
    print(f"📊 Snowpark DataFrame created with {df_snowpark.count()} rows")
        
except Exception as e:
    print(f"Error uploading data: {e}")
    print("💡 Troubleshooting tips:")
    print("   - Ensure the session is properly established")
    print("   - Check that the table exists and has the correct schema")
    print("   - Verify you have write permissions to the database/schema")


/Users/ccarrero/opt/anaconda3/envs/py310_env/lib/python3.10/site-packages/snowflake/snowpark/session.py:3290: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(


Successfully uploaded 85 rows to Snowflake table 'IMAGES_TABLE'
✅ Data upload completed using Snowpark session
📊 Snowpark DataFrame created with 85 rows


In [13]:
session.sql(f"select * from {TABLE_NAME}").show(5)

--------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"IMAGE_URL"                                         |"SOURCE_SITE"  |"CREATED_AT"                |"PROCESSED_AT"  |"STATUS"  |
--------------------------------------------------------------------------------------------------------------------------------------
|1     |https://images.unsplash.com/photo-1505740420928...  |demo_images    |2025-09-29 07:41:24.842000  |NULL            |PENDING   |
|2     |https://images.unsplash.com/photo-1511707171634...  |demo_images    |2025-09-29 07:41:24.842000  |NULL            |PENDING   |
|3     |https://images.unsplash.com/photo-1560472354-b3...  |demo_images    |2025-09-29 07:41:24.842000  |NULL            |PENDING   |
|4     |https://images.unsplash.com/photo-1550009158-9e...  |demo_images    |2025-09-29 07:41:24.842000  |NULL            |PENDING   |
|5     |https://images.unsplash.com/photo-1593642702821

### Stage area to store Images. This could be also an external S3 location



In [14]:
session.sql("CREATE OR REPLACE STAGE IMAGES ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );").collect()

session.sql("CREATE OR REPLACE STAGE FUNCTIONS;").collect()

[Row(status='Stage area FUNCTIONS successfully created.')]

Helper function to extract file name from URL:

In [15]:
import os
from urllib.parse import urlparse

def extract_name_from_url(url: str) -> str:
    parsed_url = urlparse(url)
    path = parsed_url.path
    if not path:
        return f"Error: Invalid URL path: {url}"
    
    file_path = os.path.basename(path)
    file_path = file_path.replace("-", "_")
    
    # Add .jpeg extension if not present
    if not file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp')):
        file_path += '.jpeg'

    return file_path

# ✅ Fixed: Use keyword arguments only to avoid conflict
session.udf.register(
    func=extract_name_from_url,
    name="extract_name_from_url_udf", 
    is_permanent=True,
    replace=True,
    stage_location='FUNCTIONS'
)

# Test the function
print("UDF registered successfully!")
result = extract_name_from_url("https://hello.com/carlos-carrero.png")
print(f"Test result: {result}")

UDF registered successfully!
Test result: carlos_carrero.png


### Download Images

Function to download the images from the URLs into the stage area

In [16]:
import tempfile
import os
from urllib.parse import urlparse
import urllib.request
import requests

from urllib.request import urlopen
from snowflake.snowpark import Session
from snowflake.snowpark.files import SnowflakeFile
from snowflake.snowpark.functions import udf, col # Import the udf function

def download_images_from_table(
    session: Session,
    table_name: str,
    stage_name: str,
    output_table_name: str,
    overwrite: bool = False,
) -> str:
    """
    Downloads files from URLs in a table column and uploads them to a Snowflake stage.

    Args:
        session: The Snowpark session object.
        table_name: The name of the table containing the URLs.
        stage_name: The name of the Snowflake stage where the files will be uploaded.
        overwrite: Boolean indicating whether to overwrite the file in the stage if it already exists. Defaults to False.
        parallel: The number of parallel threads to use for uploading.  Defaults to 4.

    Returns:
        str:  Returns a message indicating success, or an error.
    """

    try:
        # 1. Create a temporary directory for downloads.  Crucial to handle multiple files.
        with tempfile.TemporaryDirectory() as temp_dir:

            # 2. Read the URLs from the table.
 
            # Select only IMAGE_URL from our simplified table
            df = session.table(table_name).select(
                col("IMAGE_URL"),
                col("ID")  # Include ID for tracking
            )

            rows = df.collect()
            output_data = []
            
            # 3.  Function to download *and* upload a single file.  This is what gets mapped.
            for row in rows:
                try:
                    url = row["IMAGE_URL"]
                    parsed_url = urlparse(url)
                    path = parsed_url.path
                    if not path:
                        return f"Error: Invalid URL path: {url}"

                    file_path = os.path.basename(path)
                    file_path = file_path.replace("-", "_")
                    
                    # Add .jpeg extension if not present
                    if not file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp')):
                        file_path += '.jpeg'
            
                    local_file_path = f"{temp_dir}/{file_path}"

                    # Download the file from the URL.
                   # try:
                   #     urllib.request.urlretrieve(url, local_file_path)
                   # except Exception as e:
                   #     raise Exception(f"Failed to download file from {url}: {e}") from e

                    response = requests.get(url, timeout=10)
                    if response.status_code == 200 and 'image' in response.headers['Content-Type']:
                        with open(local_file_path, 'wb') as f:
                            f.write(response.content)
                    else:
                        raise Exception(f"Download failed or wrong content type for URL: {url}")

                    
                    # Upload the file to the Snowflake stage.

                    try:
                        session.file.put(
                            local_file_path,
                            f"@{stage_name}/",
                            overwrite=overwrite,
                            auto_compress=False
                        )
                    except Exception as e:
                        raise Exception(f"Failed to upload file to stage '{stage_name}': {e}") from e

                    print (f"Downloaded URL: {url} to file {file_path}")

                    output_data.append([
                        row["ID"], row["IMAGE_URL"], file_path
                    ])
                    
                
                except Exception as e:
                    print (f"Error processing {url}: {e}")
    
                
            if output_data:
                new_df = session.create_dataframe(output_data,
                         schema=["SOURCE_ID", "ORIGINAL_URL", "STAGE_FILE_PATH"])

                new_df.write.mode("overwrite").save_as_table(output_table_name)

            
            return "File processing completed.  Check console for individual file results."

    except Exception as e:
        error_message = f"An error occurred: {e}"
        print(error_message)
        return error_message

Calling function to ownload the images


In [17]:
# Test the complete workflow
if len(df_clean) > 0:
    print("🎯 Ready to test the complete product images workflow!")
    print(f"📊 We have {len(df_clean)} image URLs ready for processing")
    
    # Optional: Run the download function to test
    # Uncomment the lines below to actually download images to Snowflake stage
    
    TABLE_NAME_DOWNLOADED = TABLE_NAME + '_DOWNLOADED'
    
    print(f"\n💡 To download images to Snowflake stage, run:")
    print(f"   download_images_from_table(session, '{TABLE_NAME}', 'IMAGES', '{TABLE_NAME_DOWNLOADED}', True)")
    
    # Uncomment to execute:
    #
    download_images_from_table(session, TABLE_NAME, 'IMAGES', TABLE_NAME_DOWNLOADED, True)
    
else:
    print("⚠️ No image URLs available. Please check the image generation function.")

🎯 Ready to test the complete product images workflow!
📊 We have 85 image URLs ready for processing

💡 To download images to Snowflake stage, run:
   download_images_from_table(session, 'IMAGES_TABLE', 'IMAGES', 'IMAGES_TABLE_DOWNLOADED', True)
Downloaded URL: https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=600 to file photo_1505740420928_5e560c06d30e.jpeg
Downloaded URL: https://images.unsplash.com/photo-1511707171634-5f897ff02aa9?w=600 to file photo_1511707171634_5f897ff02aa9.jpeg
Downloaded URL: https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=600 to file photo_1560472354_b33ff0c44a43.jpeg
Downloaded URL: https://images.unsplash.com/photo-1550009158-9ebf69173e03?w=600 to file photo_1550009158_9ebf69173e03.jpeg
Downloaded URL: https://images.unsplash.com/photo-1593642702821-c8da6771f0c6?w=600 to file photo_1593642702821_c8da6771f0c6.jpeg
Downloaded URL: https://images.unsplash.com/photo-1484704849700-f032a568e944?w=600 to file photo_1484704849700_f032a568e944

Verify the stage area content

In [18]:
session.sql("ls @IMAGES/").collect()

[Row(name='images/photo_1434389677669_e08b4cac3105.jpeg', size=101549, md5='43dab96e30f38a37b89b318e43da9136', last_modified='Mon, 29 Sep 2025 14:50:27 GMT'),
 Row(name='images/photo_1484704849700_f032a568e944.jpeg', size=35784, md5='f1fa5a6d9f952367ecf261b1b2fb6d09', last_modified='Mon, 29 Sep 2025 14:50:11 GMT'),
 Row(name='images/photo_1498049794561_7780e7231661.jpeg', size=21382, md5='ad8c49648201f8f975f779f2fbe565ed', last_modified='Mon, 29 Sep 2025 14:50:12 GMT'),
 Row(name='images/photo_1501594907352_04cda38ebc29.jpeg', size=56304, md5='39fc214513091414b2df772c08a98be4', last_modified='Mon, 29 Sep 2025 14:50:50 GMT'),
 Row(name='images/photo_1505740420928_5e560c06d30e.jpeg', size=30799, md5='0bd0696c1a6bdc0be86fc43a19384f22', last_modified='Mon, 29 Sep 2025 14:50:07 GMT'),
 Row(name='images/photo_1506439773649_6e0eb8cfb237.jpeg', size=148810, md5='5ca66db582f23d44cb9e334b41dbac71', last_modified='Mon, 29 Sep 2025 14:50:49 GMT'),
 Row(name='images/photo_1507003211169_0a1dd7228f2d

In [19]:
TABLE_NAME_DOWNLOADED = TABLE_NAME + '_DOWNLOADED'
session.sql(f"select * from {TABLE_NAME_DOWNLOADED}").show(5)

------------------------------------------------------------------------------------------------------------
|"SOURCE_ID"  |"ORIGINAL_URL"                                      |"STAGE_FILE_PATH"                      |
------------------------------------------------------------------------------------------------------------
|1            |https://images.unsplash.com/photo-1505740420928...  |photo_1505740420928_5e560c06d30e.jpeg  |
|2            |https://images.unsplash.com/photo-1511707171634...  |photo_1511707171634_5f897ff02aa9.jpeg  |
|3            |https://images.unsplash.com/photo-1560472354-b3...  |photo_1560472354_b33ff0c44a43.jpeg     |
|4            |https://images.unsplash.com/photo-1550009158-9e...  |photo_1550009158_9ebf69173e03.jpeg     |
|5            |https://images.unsplash.com/photo-1593642702821...  |photo_1593642702821_c8da6771f0c6.jpeg  |
------------------------------------------------------------------------------------------------------------



### Building Vectors for each Image

We are going to define the table where we store the images location and the vector that represent that image




Refresh the stage area to make sure the directory table is updated

In [25]:
session.sql("alter stage images refresh").collect()

[]

#### Create AI Vector Embeddings Table

Generate vector embeddings for all images using Snowflake's AI_EMBED function with the voyage-multimodal-3 model, joining image files with their metadata.


In [26]:
# Create vectorization table with AI_EMBED processing

TABLE_NAME_DOWNLOADED_VECT = TABLE_NAME_DOWNLOADED + '_VECT'

print(f"🧠 Creating AI vectorization table: {TABLE_NAME_DOWNLOADED_VECT}")
print(f"📊 Source tables: {TABLE_NAME_DOWNLOADED} + DIRECTORY(@IMAGES)")

try:
    session.sql(f"""
        CREATE OR REPLACE TABLE {TABLE_NAME_DOWNLOADED_VECT} AS (
            WITH
            downloaded_files AS (
                SELECT 
                    SOURCE_ID,
                    ORIGINAL_URL,
                    STAGE_FILE_PATH
                FROM {TABLE_NAME_DOWNLOADED}
            ),
            stage_files AS (
                SELECT 
                    RELATIVE_PATH,
                    TO_FILE(FILE_URL) AS file_image 
                FROM DIRECTORY(@IMAGES)
            ),
            images_to_classify AS (
                SELECT 
                    d.SOURCE_ID,
                    d.ORIGINAL_URL,
                    d.STAGE_FILE_PATH,
                    s.file_image
                FROM downloaded_files d
                INNER JOIN stage_files s 
                    ON d.STAGE_FILE_PATH = s.RELATIVE_PATH
            )
            SELECT
                SOURCE_ID,
                ORIGINAL_URL,
                STAGE_FILE_PATH,
                AI_EMBED('voyage-multimodal-3', file_image) AS image_vector,
                CURRENT_TIMESTAMP() AS vector_created_at
            FROM images_to_classify
        )
    """).collect()
    
    print(f"✅ Successfully created vectorization table!")
    
    # Verify the results
    result = session.sql(f"SELECT COUNT(*) as total_vectors FROM {TABLE_NAME_DOWNLOADED_VECT}").collect()
    vector_count = result[0][0]
    print(f"📊 Total vectors created: {vector_count}")
    
    # Show sample results
    print(f"\n🔍 Sample vectorized records:")
    session.sql(f"""
        SELECT 
            *
        FROM {TABLE_NAME_DOWNLOADED_VECT} 
        LIMIT
    """).show(3)
    
except Exception as e:
    print(f"❌ Error creating vectorization table: {e}")
    print("\n🔧 Troubleshooting:")
    print("1. Check that IMAGES stage contains files")
    print("2. Verify AI_EMBED function is available")
    print("3. Ensure voyage-multimodal-3 model is accessible")


🧠 Creating AI vectorization table: IMAGES_TABLE_DOWNLOADED_VECT
📊 Source tables: IMAGES_TABLE_DOWNLOADED + DIRECTORY(@IMAGES)
✅ Successfully created vectorization table!
📊 Total vectors created: 61

🔍 Sample vectorized records:
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SOURCE_ID"  |"ORIGINAL_URL"                                      |"STAGE_FILE_PATH"                      |"IMAGE_VECTOR"                                      |"VECTOR_CREATED_AT"               |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1            |https://images.unsplash.com/photo-1505740420928...  |photo_1505740420928_5e560c06d30e.jpeg  |[0.032958984375, -0.0277099609375, 0.0295410156...  |2025-09-29 07:59:59.

Define the Cortex Search Service where we are providing the IMAGE_VECTOR column as VECTOR INDEXES. This column represent the vectos we have created (Bring Your Own Vectors)

In [27]:
session.sql(f"""
CREATE OR REPLACE CORTEX SEARCH SERVICE IMAGES_CS_SERVICE
  TEXT INDEXES ORIGINAL_URL
  VECTOR INDEXES IMAGE_VECTOR
  WAREHOUSE = 'COMPUTE_WH'
  TARGET_LAG = '1 days'
AS 
    select source_id, original_url, stage_file_path, image_vector, vector_created_at
    from {TABLE_NAME_DOWNLOADED_VECT}
;
"""
).collect()

[Row(status='Cortex search service IMAGES_CS_SERVICE successfully created.')]

Helper function to query the Cortex Search Service

In [28]:
from snowflake.core import Root

def query_cortex_search(query_text, query_vector, top_k):
    """
    Query the multi-index Cortex Search Service with emphasis on vector similarity.
    
    Args:
        query_text: Text query for the text index
        query_vector: Vector embedding for the vector index  
        top_k: Number of results to return
    
    Returns:
        Search results from the Cortex Search Service
    """
    
    root = Root(session)
    # fetch service - Updated for DEV database
    my_service = (root
      .databases["CC_IMAGES_POC"]
      .schemas["PUBLIC"]
      .cortex_search_services["IMAGES_CS_SERVICE"]
    )
    
    # Query service using multi-index syntax with vector emphasis
    resp = my_service.search(
        multi_index_query={
            # Assuming text index is on a text field (adjust column name as needed)
            # Vector index query
            "image_vector": [{"vector": query_vector}]
        },
        columns=["source_id", "original_url", "stage_file_path"],  # Specify columns to return
        limit=top_k
    )
    
    # Return all results
    return resp.results

### Test Image Similarity Search

Test the complete image similarity search workflow by using a sample image vector to find the top 5 most similar images in the collection.

In [29]:
# Test the query_cortex_search function with sample data
print("🔍 Testing Cortex Search with sample data from vectorization table...")

try:
    # Get one row from the vectorization table to use as query
    sample_query = session.sql(f"""
        SELECT 
            ORIGINAL_URL,
            IMAGE_VECTOR,
            SOURCE_ID,
            STAGE_FILE_PATH
        FROM {TABLE_NAME_DOWNLOADED_VECT} 
        LIMIT 1
    """).collect()
    
    if sample_query:
        sample_row = sample_query[0]
        query_url = sample_row['ORIGINAL_URL']
        query_vector = sample_row['IMAGE_VECTOR']
        query_source_id = sample_row['SOURCE_ID']
        query_file_path = sample_row['STAGE_FILE_PATH']
        
        print(f"📍 Using sample query:")
        print(f"   Source ID: {query_source_id}")
        print(f"   URL: {query_url}")
        print(f"   File: {query_file_path}")
        print(f"   Vector dimensions: {len(query_vector)}")
        
        # Call the query_cortex_search function
        print(f"\n🚀 Searching for top 10 similar images...")
        
        search_results = query_cortex_search(
            query_text=query_url,  # Use URL as text query
            query_vector=query_vector,  # Use the image vector
            top_k=5
        )
        
        print(f"✅ Found {len(search_results)} results!")
        
        print (search_results)
        # Display results summary
        print(f"\n📊 Search Results Summary:")
        for i, result in enumerate(search_results, 1):
            print(f"   {i}. Source ID: {result.get('source_id', 'N/A')}")
            print(f"      URL: {result.get('original_url', 'N/A')[:80]}...")
            print(f"      File: {result.get('stage_file_path', 'N/A')}")
            print()
        
        # Store results for next cell
        globals()['cortex_search_results'] = search_results
        globals()['query_info'] = {
            'source_id': query_source_id,
            'url': query_url,
            'file_path': query_file_path
        }
        
    else:
        print("❌ No data found in vectorization table")
        
except Exception as e:
    print(f"❌ Error testing Cortex Search: {e}")
    print("\n🔧 Troubleshooting:")
    print("1. Ensure the vectorization table exists and has data")
    print("2. Check that the Cortex Search Service is properly configured")
    print("3. Verify the multi-index query syntax is correct")


🔍 Testing Cortex Search with sample data from vectorization table...
📍 Using sample query:
   Source ID: 1
   URL: https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=600
   File: photo_1505740420928_5e560c06d30e.jpeg
   Vector dimensions: 1024

🚀 Searching for top 10 similar images...
✅ Found 5 results!
[{'@scores': {'cosine_similarity': 0.9409422}, 'stage_file_path': 'photo_1505740420928_5e560c06d30e.jpeg', 'original_url': 'https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=600', 'source_id': '1'}, {'@scores': {'cosine_similarity': 0.6942936}, 'stage_file_path': 'photo_1583394838336_acd977736f90.jpeg', 'original_url': 'https://images.unsplash.com/photo-1583394838336-acd977736f90?w=600', 'source_id': '8'}, {'@scores': {'cosine_similarity': 0.6385515}, 'stage_file_path': 'photo_1484704849700_f032a568e944.jpeg', 'original_url': 'https://images.unsplash.com/photo-1484704849700-f032a568e944?w=600', 'source_id': '6'}, {'@scores': {'cosine_similarity': 0.5356945}, 'st